# GENOMIC PREDICTION FOR LONGITUDINAL RESIDUAL FEED INTAKE -

## **COMPUTING CORRECTED PHENOTYPES**






This program is used to compute the corrected phenotypes obtained with different models Random regression, SAD and Multiple-trait models

In [175]:
# Load required packages
library(pacman)
pacman::p_load(dplyr,reshape2, stringr, plyr, psych, corrplot,weights,wCorr,formattable,PerformanceAnalytics)




# set path work drive
#path=getwd()
setwd("C:/Users/HP/OneDrive/RFI_all/RFI_all")




In [176]:
# pedigree
ped <- read.table("geneal.gen", quote="\"", comment.char="")
names(ped)= c("animal","pere","mere")
head(ped,10)

,animal,pere,mere
,<fct>,<fct>,<fct>
1,17M3019830268,0,0
2,17M3019830280,0,0
3,17M3019830492,0,0
4,17M3019830508,0,0
5,17M3019840010,17M3019830280,17M3019830268
6,17M3019840036,17M3019830508,17M3019830492
7,0000019851067,0,0
8,0000019858909,0,0
9,0000019858971,0,0


In [216]:
# List animals with genotypes
list_genotype <- read.table("list", quote="\"")
list_genotype$g = 1
list_genotype$animal=list_genotype[,2]
list_g=list_genotype[,c("animal","g")]
head(list_genotype)

,V1,V2,g,animal
,<int>,<fct>,<dbl>,<fct>
1,0,17MAG20004838,1,17MAG20004838
2,1,17MAG20004841,1,17MAG20004841
3,2,17MAG20004842,1,17MAG20004842
4,3,17MAG20004965,1,17MAG20004965
5,4,17MAG20005004,1,17MAG20005004
6,5,17MAG20005005,1,17MAG20005005


**Genotyoed animals by generation and line**

In [225]:
# Genotyoed animals by generation and line
ped_all <- read.csv("pedigreeRFIall", header=F, sep="") # pedigree by generation, line
names(ped_all)<-c("N","lignee","gener", "animal","p","m")
df_test=merge(ped_all,list_genotype,c("animal"))
formattable(with(df_test, table(lignee, gener)))

      gener
lignee G0 G1 G2 G3 G4 G5 G6 G7 G8 G9
     -  0 41 46 46 49 48 47  6  0  0
     +  0 45 48 49 41 54 41  6  0  0
     n 93  0  0  0  0  0  0  0  0  0

**Phenotypes**

In [177]:
# Read data 
pheno <- read.table("pheno1.dat", quote="\"", comment.char="")


# renames columns
names(pheno)<-c("animal","identif" ,"bande",
                "sex" ,"lignee"  ,"gener",
                "loge"  , "eleve","pds_dc",  
                "pds_fc" ,  "age" ,  "BW","BWpre",
                "age_dc",
                "temps","FCR","ADG","AMBW"  ,
                "BF","FI" ,"RFI","week","pe1",
                "pe2","pe3",
                "selec1","selec2","e_RR",
                "e_RRss","e_SAD","e_SADss","e_MTA","e_MTH")

pheno <- select(pheno, -c(pe1,pe3)) # drop unnecessary columns

This dataset contains the following information:
- animal, identif = id of animal
- BW = Body weight
- BWpre = predicted BW using a linear interpolation
- temps = time (in week, from week 4 to week 13 from the beginning of the test)
- pe1 = period 1 (week 1-2)
- pe2 = period 2 (week 3-5)
- e_  = residual effects


Display the first lines of dataset 'pheno'

In [227]:
head(pheno)

,animal,identif,bande,sex,lignee,gener,loge,eleve,pds_dc,pds_fc,...,week,pe2,selec1,selec2,e_RR,e_RRss,e_SAD,e_SADss,e_MTA,e_MTH
,<fct>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,<int>,<int>,...,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20005072,17MAG20005072,200011,2,n,G0,72,17MAG,24,128,...,1,1,0,1,-31.580,-30.980,-0.0009618,-9.739e-04,-28.78,-28.560
2,17MAG20005072,17MAG20005072,200011,2,n,G0,72,17MAG,24,128,...,2,1,0,1,-4.791,-4.344,-0.0001196,-9.429e-05,-1.39,-1.137
3,17MAG20005072,17MAG20005072,200011,2,n,G0,72,17MAG,24,128,...,3,2,0,1,16.910,17.230,0.0002545,2.841e-04,19.38,20.080
4,17MAG20005072,17MAG20005072,200011,2,n,G0,72,17MAG,24,128,...,4,2,0,1,16.470,16.690,0.0006243,6.332e-04,18.52,19.210
5,17MAG20005072,17MAG20005072,200011,2,n,G0,72,17MAG,24,128,...,5,2,0,1,-60.050,-59.900,-0.0016420,-1.647e-03,-58.44,-57.750
6,17MAG20005072,17MAG20005072,200011,2,n,G0,72,17MAG,24,128,...,6,3,0,1,13.970,14.090,0.0004870,4.940e-04,11.81,11.600


In above file contains data of 2397 animals, including batch(bande), sex, line (lignee), loge, eleve, BW at the begining and BW_at the end of test

In [228]:
length(unique(pheno$animal))

[1] 2397

Animals with both phenotypes and genotypes

In [229]:
length(intersect(list_genotype$animal , unique(pheno$animal)))
# 81 animals with both phenotypes and genotypes

[1] 81

 # Random regression models with matrix A

In [180]:
# Read data (outputs of Asreml program)
sln_RRAe <- read.table("ide_RRA",quote="\"") # Permanental random effect
sln_RRAg <- read.table("sln_RRA",quote="\"") # Genetic random effect
sln_RRAg$V3=sln_RRAe$V3 + sln_RRAg$V3

In [181]:
sln_RRAg = cbind(sln_RRAg,as.data.frame(matrix(unlist( strsplit(as.character(sln_RRAg$V2)  , "\\.")) , ncol = 2 , byrow = TRUE )))
names(sln_RRAg)<-c("kk","a","para","SE","gr","animal")
head(sln_RRAg)

,kk,a,para,SE,gr,animal
,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>
1,"leg(temps,2).animal",1.17M3019830268,-0.0459300,8.166,1,17M3019830268
2,"leg(temps,2).animal",1.17M3019830280,0.0009301,8.166,1,17M3019830280
3,"leg(temps,2).animal",1.17M3019830492,-0.0234300,8.166,1,17M3019830492
4,"leg(temps,2).animal",1.17M3019830508,-0.0234300,8.166,1,17M3019830508
5,"leg(temps,2).animal",1.17M3019840010,-0.0215700,8.166,1,17M3019840010
6,"leg(temps,2).animal",1.17M3019840036,-0.0468600,8.166,1,17M3019840036


In [182]:
sln_RRAg$para1=ifelse(sln_RRAg$gr==1,sln_RRAg$para,0)  
sln_RRAg$para2=ifelse(sln_RRAg$gr==2,sln_RRAg$para,0) 
sln_RRAg$para3=ifelse(sln_RRAg$gr==3,sln_RRAg$para,0)  
sln_RRAg=arrange(sln_RRAg,animal,gr)  
head(sln_RRAg)

,kk,a,para,SE,gr,animal,para1,para2,para3
,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
1,"leg(temps,2).animal",1.0000019851067,0.019560,8.166,1,0000019851067,0.01956,0.00000,0.000000
2,"leg(temps,2).animal",2.0000019851067,0.035280,7.498,2,0000019851067,0.00000,0.03528,0.000000
3,"leg(temps,2).animal",3.0000019851067,0.006404,4.539,3,0000019851067,0.00000,0.00000,0.006404
4,"leg(temps,2).animal",1.0000019858909,-0.152500,8.165,1,0000019858909,-0.15250,0.00000,0.000000
5,"leg(temps,2).animal",2.0000019858909,-0.161000,7.497,2,0000019858909,0.00000,-0.16100,0.000000
6,"leg(temps,2).animal",3.0000019858909,-0.025930,4.538,3,0000019858909,0.00000,0.00000,-0.025930


In [183]:
tatemp <- read.table("OP.txt",quote="\"") 
names(tatemp)[1]<-"temps"  
names(tatemp)[2]<-"f1"  
names(tatemp)[3]<-"f2"  
names(tatemp)[4]<-"f3"  
Temp=t(as.matrix(tatemp[2:4]))


In [184]:
# matix T
head(Temp)

f1,0.70711,0.70711,0.70711,0.70711,0.70711,0.70711,0.70711,0.70711,0.70711,0.70711
f2,-1.22474,-0.95258,-0.68041,-0.40825,-0.13608,0.13608,0.40825,0.68041,0.95258,1.22474
f3,1.58114,0.64417,-0.05856,-0.52705,-0.76129,-0.76129,-0.52705,-0.05856,0.64417,1.58114


In [185]:
#sln1=merge(sln,pheno,by = "animal")

SLN=sln_RRAg[,c("animal","gr","para")]
SLN1=arrange(SLN,animal,gr)
#SLN2=reshape(SLN1, timevar = "gr",idvar = "animal",direction="wide",sep="")
SLN2=reshape(SLN1,varying = NULL, timevar = "gr",idvar = "animal",direction="wide",sep="")
SLN3=arrange(SLN2,animal)




SLN4=SLN3[,2:4]
SLN5=as.matrix(SLN4)
EBV_all=as.data.frame((SLN5)%*%(Temp))
EBV_all$animal=SLN2$animal


df_effect=melt(EBV_all, id.vars = "animal", 
               measure.vars = c("V1", "V2","V3",
                                "V4","V5","V6","V7","V8","V9","V10"))

df_effect = arrange(df_effect,animal,variable)
df_effect$week=c(rep(1:10,3986))

In [186]:
head(df_effect)

,animal,variable,value,week
,<fct>,<fct>,<dbl>,<int>
1,0000019851067,V1,-0.019252135,1
2,0000019851067,V2,-0.015650686,2
3,0000019851067,V3,-0.010548811,3
4,0000019851067,V4,-0.003947217,4
5,0000019851067,V5,0.004154868,5
6,0000019851067,V6,0.013756673,6


In [187]:
pheno1=merge(df_effect,pheno,by=c("animal","week"))
pheno2=arrange(pheno1,animal,week)
pheno2$y_RR = pheno2$e_RR +pheno2$value

pheno2=subset(pheno2,select=-c(variable,value))
write.table(pheno2,"dat_correct_RRA")

In [188]:
head(pheno2)
summary(pheno2)

,animal,week,identif,bande,sex,lignee,gener,loge,eleve,pds_dc,...,pe2,selec1,selec2,e_RR,e_RRss,e_SAD,e_SADss,e_MTA,e_MTH,y_RR
,<fct>,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,<int>,...,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20005070,2,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,1,0,1,74.510,74.850,1.718e-03,1.719e-03,74.170,74.060,94.97610
2,17MAG20005070,4,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,2,0,1,67.060,67.430,1.574e-03,1.593e-03,63.490,64.150,79.52834
3,17MAG20005070,5,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,2,0,1,-18.340,-17.980,-6.276e-04,-6.220e-04,-27.610,-26.970,-11.37910
4,17MAG20005070,6,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,3,0,1,-29.070,-28.730,-4.628e-04,-4.519e-04,-17.430,-17.220,-28.62189
5,17MAG20005070,7,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,3,0,1,-6.171,-5.867,1.343e-05,2.321e-05,-2.308,-2.115,-13.24151
6,17MAG20005070,8,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,3,0,1,-8.770,-8.502,-1.026e-04,-1.043e-04,-13.520,-13.330,-24.36453


           animal           week                 identif          bande       
 17MAG20005072:   10   Min.   : 1.000   17MAG20005072:   10   Min.   :200011  
 17MAG20005146:   10   1st Qu.: 3.000   17MAG20005146:   10   1st Qu.:200406  
 17MAG20005197:   10   Median : 5.000   17MAG20005197:   10   Median :200612  
 17MAG20005251:   10   Mean   : 5.468   17MAG20005251:   10   Mean   :200613  
 17MAG20005297:   10   3rd Qu.: 8.000   17MAG20005297:   10   3rd Qu.:200812  
 17MAG20005301:   10   Max.   :10.000   17MAG20005301:   10   Max.   :201111  
 (Other)      :22893                    (Other)      :22893                   
      sex        lignee        gener           loge          eleve      
 Min.   :1.000   -:11744   G7     :4970   Min.   : 71.00   17MAG:13355  
 1st Qu.:2.000   +:10456   G6     :3748   1st Qu.: 81.00   86001: 9598  
 Median :2.000   n:  753   G5     :3218   Median : 91.00                
 Mean   :2.016             G4     :3107   Mean   : 87.89                
 3r

## Random regression models with matrix H

In [189]:
# Random regression model with matrix H
#rm(list = ls()) # ecrase previous memories
###############################################################################################
pheno2=read.table("dat_correct_RRA")


sln_RRHe <- read.table("ide_RRH",quote="\"") # Permanental random effect
sln_RRHg <- read.table("sln_RRH",quote="\"") # Genetic random effect
sln_RRHg$V3=sln_RRHe$V3 + sln_RRHg$V3

# sln <- read.table("C:/Users/vhtran/Google Drive/these/RFI/INRA/validation/ide_RRA",quote="\"") # 
# sln_1 <- read.table("C:/Users/vhtran/Google Drive/these/RFI/INRA/validation/sln_RRA",quote="\"") 
m_RRH <- as.data.frame(matrix( unlist( strsplit(as.character(sln_RRHg$V2)  , "\\." ) ) , ncol = 2 , byrow = TRUE ))
sln_RRHg=cbind(sln_RRHg,m_RRH)
names(sln_RRHg)<-c("kk","a","para","SE","gr","animal")



In [190]:
head(sln_RRHg)

,kk,a,para,SE,gr,animal
,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>
1,"leg(temps,2).giv(ani",1.17M3019830268,-0.11520,8.057,1,17M3019830268
2,"leg(temps,2).giv(ani",1.17M3019830280,-0.00814,8.057,1,17M3019830280
3,"leg(temps,2).giv(ani",1.17M3019830492,-0.05354,8.057,1,17M3019830492
4,"leg(temps,2).giv(ani",1.17M3019830508,-0.05354,8.057,1,17M3019830508
5,"leg(temps,2).giv(ani",1.17M3019840010,-0.06982,8.057,1,17M3019840010
6,"leg(temps,2).giv(ani",1.17M3019840036,-0.10710,8.057,1,17M3019840036


In [191]:
sln_RRHg$para1=ifelse(sln_RRHg$gr==1,sln_RRHg$para,0)  
sln_RRHg$para2=ifelse(sln_RRHg$gr==2,sln_RRHg$para,0) 
sln_RRHg$para3=ifelse(sln_RRHg$gr==3,sln_RRHg$para,0)  
sln_RRHg=arrange(sln_RRHg,animal,gr)  
head(sln_RRHg)


,kk,a,para,SE,gr,animal,para1,para2,para3
,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
1,"leg(temps,2).giv(ani",1.0000019851067,0.018410,8.057,1,0000019851067,0.01841,0.00000,0.000000
2,"leg(temps,2).giv(ani",2.0000019851067,0.031990,7.319,2,0000019851067,0.00000,0.03199,0.000000
3,"leg(temps,2).giv(ani",3.0000019851067,0.004749,3.771,3,0000019851067,0.00000,0.00000,0.004749
4,"leg(temps,2).giv(ani",1.0000019858909,-0.243100,8.059,1,0000019858909,-0.24310,0.00000,0.000000
5,"leg(temps,2).giv(ani",2.0000019858909,-0.397000,7.320,2,0000019858909,0.00000,-0.39700,0.000000
6,"leg(temps,2).giv(ani",3.0000019858909,0.016640,3.771,3,0000019858909,0.00000,0.00000,0.016640


In [192]:
SLN_RRH=sln_RRHg[,c("animal","gr","para")]
SLN_RRH1=arrange(SLN_RRH,animal,gr)
#SLN2=reshape(SLN1, timevar = "gr",idvar = "animal",direction="wide",sep="")
SLN_RRH2=reshape(SLN_RRH1,varying = NULL, timevar = "gr",idvar = "animal",direction="wide",sep="")
SLN_RRH3=arrange(SLN_RRH2,animal)



SLN_RRH4=SLN_RRH3[,2:4]
SLN_RRH5=as.matrix(SLN_RRH4)
EBV_all1=as.data.frame((SLN_RRH5)%*%(Temp))
EBV_all1$animal=SLN_RRH2$animal

In [193]:
head(EBV_all1)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,animal
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,-0.01865270,-0.01439598,-0.009026522,-0.002544983,0.00504933,0.01375573,0.02357485,0.03450611,0.04655009,0.05970616,0000019851067
2,0.34063351,0.21699481,0.097249891,-0.018593303,-0.13054255,-0.23859007,-0.34274380,-0.44299565,-0.53935371,-0.63181005,0000019858909
3,-0.01865270,-0.01439598,-0.009026522,-0.002544983,0.00504933,0.01375573,0.02357485,0.03450611,0.04655009,0.05970616,0000019858971
4,0.05959042,0.03349862,0.010659931,-0.008924683,-0.02525605,-0.03833333,-0.04815751,-0.05472747,-0.05804432,-0.05810709,0000019859155
5,0.26493798,0.21916376,0.175319683,0.133408547,0.09342764,0.05537967,0.01926185,-0.01492295,-0.04717761,-0.07749932,0000019860769
6,0.26493798,0.21916376,0.175319683,0.133408547,0.09342764,0.05537967,0.01926185,-0.01492295,-0.04717761,-0.07749932,0000019860909


In [194]:
df_effect1=melt(EBV_all1, 
               id.vars = "animal", 
               measure.vars = c("V1", "V2","V3","V4","V5","V6","V7","V8","V9","V10"))

df_effect1= arrange(df_effect1,animal,variable)
df_effect1$week=c(rep(1:10,3986))

# combine the last dataset pheno2 with EBV from RR with matrix H
pheno3=merge(df_effect1,pheno2,by=c("animal","week"))
pheno4=arrange(pheno3,animal,week)
pheno4$y_RRH = pheno4$e_RRss+pheno4$value
pheno4=subset(pheno4,select=-c(variable,value))

#write.table(pheno4,"dat_correct_RRA_RRH")

In [195]:
head(pheno4)

,animal,week,identif,bande,sex,lignee,gener,loge,eleve,pds_dc,...,selec1,selec2,e_RR,e_RRss,e_SAD,e_SADss,e_MTA,e_MTH,y_RR,y_RRH
,<fct>,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,<int>,...,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20005070,2,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,0,1,74.510,74.850,1.718e-03,1.719e-03,74.170,74.060,94.97610,95.87767
2,17MAG20005070,4,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,0,1,67.060,67.430,1.574e-03,1.593e-03,63.490,64.150,79.52834,79.07920
3,17MAG20005070,5,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,0,1,-18.340,-17.980,-6.276e-04,-6.220e-04,-27.610,-26.970,-11.37910,-12.41159
4,17MAG20005070,6,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,0,1,-29.070,-28.730,-4.628e-04,-4.519e-04,-17.430,-17.220,-28.62189,-30.16971
5,17MAG20005070,7,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,0,1,-6.171,-5.867,1.343e-05,2.321e-05,-2.308,-2.115,-13.24151,-15.24268
6,17MAG20005070,8,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,0,1,-8.770,-8.502,-1.026e-04,-1.043e-04,-13.520,-13.330,-24.36453,-26.74102


## SAD models with matrix A

In [196]:
###############################################################################################
pheno4=read.table("dat_correct_RRA_RRH")



sln_sad <- read.table("sln_sadA", quote="\"") 
ide_sad<- read.table("ide_sadA", quote="\"")
sln_sad$ide_sad = ide_sad$V3


sln_sad$somme= sln_sad$V3 + sln_sad$ide_sad
names(sln_sad)<-c("V1","code","g","SEg","e","somme")



m_sadA <- as.data.frame(matrix( unlist( strsplit(as.character(sln_sad$code)  , "\\." ) ) , ncol = 2 , byrow = TRUE ))

sln_sad1=cbind(sln_sad,m_sadA)
names(sln_sad1)<-c("V1","code","g","SEgA","e","somme","temps","animal")
sln_sad1$temps=as.numeric(levels(sln_sad1$temps))[sln_sad1$temps]
sln_sad2=arrange(sln_sad1,animal,temps)
N_animal=length(unique(sln_sad2$animal))  #number animal from SAD.sln



sln_sad2$week=sln_sad2$temps
sln_sad2=subset(sln_sad2,select=-c(temps))
critere=c("animal","week")
DF_s=merge(pheno4,sln_sad2,by=c(critere))
DF_s$y_SAD = DF_s$e_SAD + DF_s$somme

DF_s <- select(DF_s, -c(somme,code, V1,g,e))

head(DF_s)
write.table(DF_s,"dat_correct_RRA_RRH_sadA")
############################################################


,animal,week,identif,bande,sex,lignee,gener,loge,eleve,pds_dc,...,e_RR,e_RRss,e_SAD,e_SADss,e_MTA,e_MTH,y_RR,y_RRH,SEgA,y_SAD
,<fct>,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20005070,10,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,-1.120,-0.9432,-2.249e-04,-2.081e-04,-5.578,-5.368,-36.77958,-39.69193,11.780,-36.55022
2,17MAG20005070,2,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,74.510,74.8500,1.718e-03,1.719e-03,74.170,74.060,94.97610,95.87767,8.846,92.79172
3,17MAG20005070,4,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,67.060,67.4300,1.574e-03,1.593e-03,63.490,64.150,79.52834,79.07920,7.179,78.04357
4,17MAG20005070,5,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,-18.340,-17.9800,-6.276e-04,-6.220e-04,-27.610,-26.970,-11.37910,-12.41159,7.112,-11.63063
5,17MAG20005070,6,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,-29.070,-28.7300,-4.628e-04,-4.519e-04,-17.430,-17.220,-28.62189,-30.16971,7.418,-29.23046
6,17MAG20005070,7,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,-6.171,-5.8670,1.343e-05,2.321e-05,-2.308,-2.115,-13.24151,-15.24268,8.000,-14.00499


In [197]:
###################################################
#rm(list = ls())


###############################################################################################
DF_s=read.table("dat_correct_RRA_RRH_sadA")

sln_sadH <- read.table("sln_sadH", quote="\"") #old is G_sad
ide_sadH <- read.table("ide_sadH", quote="\"")
sln_sadH$ide_sad = ide_sadH$V3

sln_sadH$somme= sln_sadH$V3 + sln_sadH$ide_sad
names(sln_sadH)<-c("V1","code","g","SEg","e","somme")

m_sadH <- as.data.frame(matrix(unlist( strsplit(as.character(sln_sadH$code), "\\." )) , ncol = 2 , byrow = TRUE ))


sln_sadH1=cbind(sln_sadH,m_sadH)
names(sln_sadH1)<-c("V11","code1","g1","SEgH","e1","somme1","temps","animal")

sln_sadH1$temps=as.numeric(levels(sln_sadH1$temps))[sln_sadH1$temps]
sln_sadH2=arrange(sln_sadH1,animal,temps)
N_animal=length(unique(sln_sadH2$animal))  #number animal from SAD.sln
sln_sadH2$week=sln_sadH2$temps
sln_sadH2=select(sln_sadH2,-c(temps))




critere=c("animal","week")
DF_s2 =merge(DF_s,sln_sadH2,by=c(critere))


DF_s2$y_SADH = DF_s2$e_SADss + DF_s2$somme1

# Delete some columns
DF_s2=select(DF_s2,-c(code1,somme1,V11,code1,g1,e1))
# write table
write.table(DF_s2,"dat_correct_RRA_RRH_sadA_SADH")

In [198]:
head(DF_s2)

,animal,week,identif,bande,sex,lignee,gener,loge,eleve,pds_dc,...,e_SAD,e_SADss,e_MTA,e_MTH,y_RR,y_RRH,SEgA,y_SAD,SEgH,y_SADH
,<fct>,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20005070,10,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,-2.249e-04,-2.081e-04,-5.578,-5.368,-36.77958,-39.69193,11.780,-36.55022,11.920,-39.64021
2,17MAG20005070,2,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,1.718e-03,1.719e-03,74.170,74.060,94.97610,95.87767,8.846,92.79172,8.403,92.78072
3,17MAG20005070,4,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,1.574e-03,1.593e-03,63.490,64.150,79.52834,79.07920,7.179,78.04357,7.064,76.62459
4,17MAG20005070,5,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,-6.276e-04,-6.220e-04,-27.610,-26.970,-11.37910,-12.41159,7.112,-11.63063,7.190,-13.23762
5,17MAG20005070,6,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,-4.628e-04,-4.519e-04,-17.430,-17.220,-28.62189,-30.16971,7.418,-29.23046,7.678,-31.02945
6,17MAG20005070,7,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,1.343e-05,2.321e-05,-2.308,-2.115,-13.24151,-15.24268,8.000,-14.00499,8.401,-16.34498


## Multiple trait models

In [199]:
# rm(list = ls())

# Read files
DF_s2=read.table("dat_correct_RRA_RRH_sadA_SADH")
sln_mt <- read.table("sln_mtA", quote="\"", comment.char="") #old is G_sad
ide_mt <- read.table("ide_mtA", quote="\"", comment.char="")
sln_mt$V3=sln_mt$V3+ide_mt$V3
names(sln_mt)<-c("V1","code","value","SE")





m_mtA <- as.data.frame(matrix( unlist(strsplit(as.character(sln_mt$code)  , "\\.")) , ncol = 2 , byrow = TRUE ))
sln_mt1=cbind(sln_mt,m_mtA)
names(sln_mt1)<-c("V1","code","value","SEg_MTA","pe2","animal")
sln_mt1$pe2=as.numeric(levels(sln_mt1$pe2))[sln_mt1$pe2]
sln_mt2=arrange(sln_mt1,animal,pe2)
N_animal=length(unique(sln_mt2$animal))  #number animal from SAD.sln


critere=c("animal","pe2")
DF_s3=merge(DF_s2,sln_mt2,by=c(critere))
DF_s3$y_MT = DF_s3$e_MTA+DF_s3$value # corrected phenotypes
DF_s3 = select(DF_s3,-c(value, code, V1)) # drop some columns
write.table(DF_s3,"dat_correct_RRA_RRH_sadA_SADH_mtA")
########################################################################################################

head(DF_s3)


,animal,pe2,week,identif,bande,sex,lignee,gener,loge,eleve,...,e_MTA,e_MTH,y_RR,y_RRH,SEgA,y_SAD,SEgH,y_SADH,SEg_MTA,y_MT
,<fct>,<int>,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20005070,1,2,17MAG20005070,200011,2,n,G0,71,17MAG,...,74.170,74.060,94.97610,95.87767,8.846,92.79172,8.403,92.78072,7.612,95.4500
2,17MAG20005070,2,4,17MAG20005070,200011,2,n,G0,71,17MAG,...,63.490,64.150,79.52834,79.07920,7.179,78.04357,7.064,76.62459,5.936,80.2270
3,17MAG20005070,2,5,17MAG20005070,200011,2,n,G0,71,17MAG,...,-27.610,-26.970,-11.37910,-12.41159,7.112,-11.63063,7.190,-13.23762,5.936,-10.8730
4,17MAG20005070,3,6,17MAG20005070,200011,2,n,G0,71,17MAG,...,-17.430,-17.220,-28.62189,-30.16971,7.418,-29.23046,7.678,-31.02945,6.505,-28.2649
5,17MAG20005070,3,7,17MAG20005070,200011,2,n,G0,71,17MAG,...,-2.308,-2.115,-13.24151,-15.24268,8.000,-14.00499,8.401,-16.34498,6.505,-13.1429
6,17MAG20005070,3,8,17MAG20005070,200011,2,n,G0,71,17MAG,...,-13.520,-13.330,-24.36453,-26.74102,8.805,-25.16910,9.251,-27.83010,6.505,-24.3549


In [200]:
########################################################################################################
#rm(list = ls())
DF_s3=read.table("dat_correct_RRA_RRH_sadA_SADH_mtA")
sln_mtH <- read.table("sln_mtH", quote="\"", comment.char="") #old is G_sad
ide_mtH <- read.table("ide_mtH", quote="\"", comment.char="")
sln_mtH$V3=sln_mtH$V3+ide_mtH$V3
names(sln_mtH)<-c("V1","code","value","SE")

m_mtH <- as.data.frame(matrix(unlist(strsplit(as.character(sln_mtH$code)  , "\\." )), ncol = 2 , byrow = TRUE ))
sln_mtH1=cbind(sln_mtH,m_mtH)
names(sln_mtH1)<-c("V11","code1","value1","SEg_MTH","pe2","animal")
head(sln_mtH1)
sln_mtH1 <- select(sln_mtH1,c(animal,pe2,value1,SEg_MTH))

,V11,code1,value1,SEg_MTH,pe2,animal
,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>
1,"period1.giv(animal,1",1.17M3019830268,0.10790,7.714,1,17M3019830268
2,"period1.giv(animal,1",1.17M3019830280,0.02548,7.714,1,17M3019830280
3,"period1.giv(animal,1",1.17M3019830492,0.04120,7.714,1,17M3019830492
4,"period1.giv(animal,1",1.17M3019830508,0.04120,7.714,1,17M3019830508
5,"period1.giv(animal,1",1.17M3019840010,0.09216,7.714,1,17M3019840010
6,"period1.giv(animal,1",1.17M3019840036,0.08239,7.714,1,17M3019840036


In [201]:
critere=c("animal","pe2")
DF_s4=merge(DF_s3,sln_mtH1,by=c(critere))
DF_s4$y_MTH = DF_s4$e_MTH+DF_s4$value1

DF_s4=subset(DF_s4,select=-c(value1))
write.table(DF_s4,"dat_correct_RRA_RRH_sadA_SADH_mtA_mtH")

# check the correlation between the corrected phenotypes

with(DF_s4,cor(y_RR,y_SADH))
with(DF_s4,cor(y_RRH,y_RR))
head(DF_s4)

[1] 0.9947062

[1] 0.9960819

,animal,pe2,week,identif,bande,sex,lignee,gener,loge,eleve,...,y_RR,y_RRH,SEgA,y_SAD,SEgH,y_SADH,SEg_MTA,y_MT,SEg_MTH,y_MTH
,<fct>,<int>,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20005070,1,2,17MAG20005070,200011,2,n,G0,71,17MAG,...,94.97610,95.87767,8.846,92.79172,8.403,92.78072,7.612,95.4500,6.915,96.640
2,17MAG20005070,2,4,17MAG20005070,200011,2,n,G0,71,17MAG,...,79.52834,79.07920,7.179,78.04357,7.064,76.62459,5.936,80.2270,5.822,79.649
3,17MAG20005070,2,5,17MAG20005070,200011,2,n,G0,71,17MAG,...,-11.37910,-12.41159,7.112,-11.63063,7.190,-13.23762,5.936,-10.8730,5.822,-11.471
4,17MAG20005070,3,6,17MAG20005070,200011,2,n,G0,71,17MAG,...,-28.62189,-30.16971,7.418,-29.23046,7.678,-31.02945,6.505,-28.2649,6.555,-30.280
5,17MAG20005070,3,7,17MAG20005070,200011,2,n,G0,71,17MAG,...,-13.24151,-15.24268,8.000,-14.00499,8.401,-16.34498,6.505,-13.1429,6.555,-15.175
6,17MAG20005070,3,8,17MAG20005070,200011,2,n,G0,71,17MAG,...,-24.36453,-26.74102,8.805,-25.16910,9.251,-27.83010,6.505,-24.3549,6.555,-26.390


In [202]:
sessionInfo()

R version 3.6.3 (2020-02-29)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 10 x64 (build 18363)

Matrix products: default

locale:
[1] LC_COLLATE=English_United States.1252 
[2] LC_CTYPE=English_United States.1252   
[3] LC_MONETARY=English_United States.1252
[4] LC_NUMERIC=C                          
[5] LC_TIME=English_United States.1252    

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] PerformanceAnalytics_2.0.4 xts_0.12-0                
 [3] zoo_1.8-7                  formattable_0.2.0.1       
 [5] wCorr_1.9.1                weights_1.0.1             
 [7] mice_3.8.0                 gdata_2.18.0              
 [9] Hmisc_4.3-1                ggplot2_3.3.0             
[11] Formula_1.2-3              survival_3.1-8            
[13] lattice_0.20-38            corrplot_0.84             
[15] psych_1.9.12.31            plyr_1.8.5                
[17] stringr_1.4.0              reshap

In [203]:
#ped <- read.table("geneal.gen", quote="\"", comment.char="")
#names(ped)= c("animal","pere","mere")
# Now we are tracking back the number of progenes each dam/sire has. 

In [208]:
head(ped,20)

,animal,pere,mere
,<fct>,<fct>,<fct>
1,17M3019830268,0,0
2,17M3019830280,0,0
3,17M3019830492,0,0
4,17M3019830508,0,0
5,17M3019840010,17M3019830280,17M3019830268
6,17M3019840036,17M3019830508,17M3019830492
7,0000019851067,0,0
8,0000019858909,0,0
9,0000019858971,0,0


In [209]:
ped1=arrange(ped,mere,animal)
ped2=arrange(ped,pere,animal)

In [210]:
head(ped1,20)

,animal,pere,mere
,<fct>,<fct>,<fct>
1,0000019851067,0,0
2,0000019858909,0,0
3,0000019858971,0,0
4,0000019859155,0,0
5,0000019860769,0,0
6,0000019860909,0,0
7,02JC119876571,0,0
8,03HP119903200,0,0
9,03HP119913663,0,0


In [211]:
#names(ped1)= c("progeny","animal","mari")
names(ped1)= c("progeny","mari","animal") # list classed by dam
names(ped2)= c("progeny","animal","wife") # ist classed by sire

In [212]:
head(ped1,20)

,progeny,mari,animal
,<fct>,<fct>,<fct>
1,0000019851067,0,0
2,0000019858909,0,0
3,0000019858971,0,0
4,0000019859155,0,0
5,0000019860769,0,0
6,0000019860909,0,0
7,02JC119876571,0,0
8,03HP119903200,0,0
9,03HP119913663,0,0


In [213]:
head(ped2,20)

,progeny,animal,wife
,<fct>,<fct>,<fct>
1,0000019851067,0,0
2,0000019858909,0,0
3,0000019858971,0,0
4,0000019859155,0,0
5,0000019860769,0,0
6,0000019860909,0,0
7,02JC119876571,0,0
8,03HP119903200,0,0
9,03HP119913663,0,0


In [214]:
list_genotype <- read.table("list", quote="\"")
list_genotype$g = 1
list_genotype$animal=list_genotype[,2]
list_g=list_genotype[,c("animal","g")]

head(list_genotype)

,V1,V2,g,animal
,<int>,<fct>,<dbl>,<fct>
1,0,17MAG20004838,1,17MAG20004838
2,1,17MAG20004841,1,17MAG20004841
3,2,17MAG20004842,1,17MAG20004842
4,3,17MAG20004965,1,17MAG20004965
5,4,17MAG20005004,1,17MAG20005004
6,5,17MAG20005005,1,17MAG20005005


In [215]:
list_g2= merge(ped1,list_g,c("animal")) # dams with genotypes
list_g2_p= merge(ped2,list_g,c("animal")) # sires with genotypes


In [122]:
head(list_g2,20)

,animal,progeny,mari,g
,<fct>,<fct>,<fct>,<dbl>
1,17MAG20004838,17MAG20012561,17MAG20005775,1
2,17MAG20004838,17MAG20012567,17MAG20005775,1
3,17MAG20004838,17MAG20012560,17MAG20005775,1
4,17MAG20004838,17MAG20012562,17MAG20005775,1
5,17MAG20004841,8600120010023,17MAG20005197,1
6,17MAG20004841,8600120010021,17MAG20005197,1
7,17MAG20004841,8600120020008,17MAG20005197,1
8,17MAG20004841,8600120020010,17MAG20005197,1
9,17MAG20004841,8600120010026,17MAG20005197,1


Herein, we observed that the dam 17MAG20004838 has 4 progenies : <br>
- 17MAG20012561
- 17MAG20012567
- 17MAG20012560
- 17MAG20012562

In [123]:
head(list_g2_p,28)
list_g2_p[1:23,2]

,animal,progeny,wife,g
,<fct>,<fct>,<fct>,<dbl>
1,17MAG20005073,8600120022019,17MAG20005322,1
2,17MAG20005073,8600120022110,17MAG20005316,1
3,17MAG20005073,8600120022018,17MAG20005322,1
4,17MAG20005073,8600120022015,17MAG20005322,1
5,17MAG20005073,17MAG20012765,17MAG20005554,1
6,17MAG20005073,17MAG20012768,17MAG20005554,1
7,17MAG20005073,8600120022016,17MAG20005322,1
8,17MAG20005073,8600120022111,17MAG20005316,1
9,17MAG20005073,17MAG20013046,17MAG20005601,1


[1] 8600120022019 8600120022110 8600120022018 8600120022015 17MAG20012765
 [6] 17MAG20012768 8600120022016 8600120022111 17MAG20013046 8600120012005
[11] 17MAG20012762 8600120012163 8600120012166 17MAG20012763 8600120012160
[16] 17MAG20013047 17MAG20013045 8600120012141 8600120012142 8600120012003
[21] 8600120012010 8600120012157 8600120012138
3986 Levels: 0000019851067 0000019858909 0000019858971 ... FI04319920706

Herein, we observed that the sire 17MAG20004838 has 23 progenies : <br>
- 8600120022019
- 8600120022110
- 8600120022018
- 8600120022015
- 17MAG20012765
- ...
- 8600120012138

In [124]:
length(unique((list_g2$animal)))
length(unique((list_g2_p$animal)))
names(list_g2)= c("mere","animal","pere","g") #list classed by dams
names(list_g2_p)= c("pere","animal","mere","g") #list classed by sires

[1] 568

[1] 80

- 568 dams with genotypes
- 80 sires with genotypes

In [125]:
DF_s4 <- read.table("dat_correct_RRA_RRH_sadA_SADH_mtA_mtH", header=TRUE, quote="\"")
df=merge(list_g2,DF_s4,c("animal")) # list animals with both phenotypes and genotypes


In [126]:
intersect(unique(list_g2$mere), unique(DF_s4$animal))
# No common element

character(0)

This indicates that the animal ignored own performances

In [127]:
length(unique(df$animal))
df1=arrange(df,mere,animal,week)
df2=subset(df1,!is.na(week))
#summary(df1 == df2)

[1] 2254

In [128]:
head(df1)

,animal,mere,pere,g,pe2,week,identif,bande,sex,lignee,...,y_RR,y_RRH,SEgA,y_SAD,SEgH,y_SADH,SEg_MTA,y_MT,SEg_MTH,y_MTH
,<fct>,<fct>,<fct>,<dbl>,<int>,<int>,<fct>,<int>,<int>,<fct>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20012560,17MAG20004838,17MAG20005775,1,1,1,17MAG20012560,200114,2,+,...,22.96108,25.18634,9.981,24.64040,9.649,26.01037,7.154,23.32080,6.635,24.97700
2,17MAG20012560,17MAG20004838,17MAG20005775,1,1,2,17MAG20012560,200114,2,+,...,11.00911,12.25526,8.643,10.94692,8.272,11.33094,7.154,11.25680,6.635,12.92100
3,17MAG20012560,17MAG20004838,17MAG20005775,1,2,3,17MAG20012560,200114,2,+,...,12.50314,12.86136,7.546,11.55604,7.312,10.86305,5.749,12.68341,5.681,12.13529
4,17MAG20012560,17MAG20004838,17MAG20005775,1,2,4,17MAG20012560,200114,2,+,...,16.47332,16.01383,7.031,15.96352,6.970,14.48151,5.749,16.50941,5.681,15.95229
5,17MAG20012560,17MAG20004838,17MAG20005775,1,2,5,17MAG20012560,200114,2,+,...,-31.01846,-32.21647,6.989,-30.96601,7.114,-32.74871,5.749,-31.10059,5.681,-31.64771
6,17MAG20012560,17MAG20004838,17MAG20005775,1,3,6,17MAG20012560,200114,2,+,...,21.29690,19.44859,7.303,20.37976,7.610,18.29016,6.319,21.55380,6.411,19.06100


In [129]:
head(df2)

,animal,mere,pere,g,pe2,week,identif,bande,sex,lignee,...,y_RR,y_RRH,SEgA,y_SAD,SEgH,y_SADH,SEg_MTA,y_MT,SEg_MTH,y_MTH
,<fct>,<fct>,<fct>,<dbl>,<int>,<int>,<fct>,<int>,<int>,<fct>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20012560,17MAG20004838,17MAG20005775,1,1,1,17MAG20012560,200114,2,+,...,22.96108,25.18634,9.981,24.64040,9.649,26.01037,7.154,23.32080,6.635,24.97700
2,17MAG20012560,17MAG20004838,17MAG20005775,1,1,2,17MAG20012560,200114,2,+,...,11.00911,12.25526,8.643,10.94692,8.272,11.33094,7.154,11.25680,6.635,12.92100
3,17MAG20012560,17MAG20004838,17MAG20005775,1,2,3,17MAG20012560,200114,2,+,...,12.50314,12.86136,7.546,11.55604,7.312,10.86305,5.749,12.68341,5.681,12.13529
4,17MAG20012560,17MAG20004838,17MAG20005775,1,2,4,17MAG20012560,200114,2,+,...,16.47332,16.01383,7.031,15.96352,6.970,14.48151,5.749,16.50941,5.681,15.95229
5,17MAG20012560,17MAG20004838,17MAG20005775,1,2,5,17MAG20012560,200114,2,+,...,-31.01846,-32.21647,6.989,-30.96601,7.114,-32.74871,5.749,-31.10059,5.681,-31.64771
6,17MAG20012560,17MAG20004838,17MAG20005775,1,3,6,17MAG20012560,200114,2,+,...,21.29690,19.44859,7.303,20.37976,7.610,18.29016,6.319,21.55380,6.411,19.06100


In [130]:
length(unique(df2$animal))

[1] 2254

In [131]:
df_p=merge(list_g2_p,DF_s4,c("animal"))
length(df_p$animal)
df1_p=arrange(df_p,pere,animal,week)
df2_p=subset(df1_p,!is.na(week))


df2$animal=as.character(df2$animal)
df2$mere=as.character(df2$mere)
df2$week=as.numeric(df2$week)


[1] 21208

In [132]:
length(unique(df2_p$animal))

[1] 2213

In [133]:
df3=select(df2,animal,mere,week,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH, FI, FCR, ADG, RFI)
df3 <- arrange(df3, mere, week)
df3$sub <- paste0(df3$mere,df3$week) #creat criteria for selecting unique animial-week
df3 <- arrange(df3,sub)
df3$sub

[1] "17MAG200048381"  "17MAG200048381"  "17MAG200048381"  "17MAG2000483810"
    [5] "17MAG2000483810" "17MAG2000483810" "17MAG200048382"  "17MAG200048382" 
    [9] "17MAG200048382"  "17MAG200048383"  "17MAG200048383"  "17MAG200048383" 
   [13] "17MAG200048384"  "17MAG200048384"  "17MAG200048384"  "17MAG200048385" 
   [17] "17MAG200048385"  "17MAG200048385"  "17MAG200048386"  "17MAG200048386" 
   [21] "17MAG200048386"  "17MAG200048387"  "17MAG200048387"  "17MAG200048387" 
   [25] "17MAG200048388"  "17MAG200048388"  "17MAG200048388"  "17MAG200048389" 
   [29] "17MAG200048389"  "17MAG200048389"  "17MAG200048411"  "17MAG200048411" 
   [33] "17MAG2000484110" "17MAG2000484110" "17MAG200048412"  "17MAG200048412" 
   [37] "17MAG200048413"  "17MAG200048413"  "17MAG200048414"  "17MAG200048415" 
   [41] "17MAG200048416"  "17MAG200048416"  "17MAG200048417"  "17MAG200048417" 
   [45] "17MAG200048418"  "17MAG200048418"  "17MAG200048419"  "17MAG200048419" 
   [49] "17MAG200048421"  "17MAG2000484210" "17MAG200048422"  "17MAG200048423" 
   [53] "17MAG200048424"  "17MAG200048425"  "17MAG200048426"  "17MAG200048427" 
   [57] "17MAG200048428"  "17MAG200048429"  "17MAG200049651"  "17MAG200049651" 
   [61] "17MAG2000496510" "17MAG2000496510" "17MAG200049652"  "17MAG200049652" 
   [65] "17MAG200049653"  "17MAG200049653"  "17MAG200049654"  "17MAG200049654" 
   [69] "17MAG200049655"  "17MAG200049655"  "17MAG200049656"  "17MAG200049656" 
   [73] "17MAG200049657"  "17MAG200049657"  "17MAG200049658"  "17MAG200049658" 
   [77] "17MAG200049659"  "17MAG200049659"  "17MAG200050041"  "17MAG200050041" 
   [81] "17MAG2000500410" "17MAG2000500410" "17MAG200050042"  "17MAG200050042" 
   [85] "17MAG200050043"  "17MAG200050043"  "17MAG200050044"  "17MAG200050044" 
   [89] "17MAG200050045"  "17MAG200050045"  "17MAG200050046"  "17MAG200050046" 
   [93] "17MAG200050047"  "17MAG200050047"  "17MAG200050048"  "17MAG200050048" 
   [97] "17MAG200050049"  "17MAG200050049"  "17MAG200050051"  "17MAG200050051" 
  [101] "17MAG200050051"  "17MAG2000500510" "17MAG2000500510" "17MAG2000500510"
  [105] "17MAG200050052"  "17MAG200050052"  "17MAG200050052"  "17MAG200050053" 
  [109] "17MAG200050053"  "17MAG200050053"  "17MAG200050054"  "17MAG200050054" 
  [113] "17MAG200050054"  "17MAG200050055"  "17MAG200050055"  "17MAG200050055" 
  [117] "17MAG200050056"  "17MAG200050056"  "17MAG200050056"  "17MAG200050057" 
  [121] "17MAG200050057"  "17MAG200050058"  "17MAG200050058"  "17MAG200050058" 
  [125] "17MAG200050059"  "17MAG200050059"  "17MAG200050059"  "17MAG200050751" 
  [129] "17MAG200050751"  "17MAG2000507510" "17MAG2000507510" "17MAG200050752" 
  [133] "17MAG200050752"  "17MAG200050753"  "17MAG200050753"  "17MAG200050754" 
  [137] "17MAG200050754"  "17MAG200050755"  "17MAG200050755"  "17MAG200050756" 
  [141] "17MAG200050756"  "17MAG200050757"  "17MAG200050757"  "17MAG200050758" 
  [145] "17MAG200050758"  "17MAG200050759"  "17MAG200050759"  "17MAG200050761" 
  [149] "17MAG200050761"  "17MAG2000507610" "17MAG2000507610" "17MAG200050762" 
  [153] "17MAG200050762"  "17MAG200050762"  "17MAG200050763"  "17MAG200050763" 
  [157] "17MAG200050763"  "17MAG200050764"  "17MAG200050764"  "17MAG200050764" 
  [161] "17MAG200050765"  "17MAG200050765"  "17MAG200050765"  "17MAG200050766" 
  [165] "17MAG200050766"  "17MAG200050766"  "17MAG200050767"  "17MAG200050767" 
  [169] "17MAG200050767"  "17MAG200050768"  "17MAG200050768"  "17MAG200050768" 
  [173] "17MAG200050769"  "17MAG200050769"  "17MAG200051531"  "17MAG200051531" 
  [177] "17MAG200051531"  "17MAG2000515310" "17MAG2000515310" "17MAG200051532" 
  [181] "17MAG200051532"  "17MAG200051532"  "17MAG200051533"  "17MAG200051533" 
  [185] "17MAG200051533"  "17MAG200051534"  "17MAG200051534"  "17MAG200051534" 
  [189] "17MAG200051535"  "17MAG200051535"  "17MAG200051535"  "17MAG200051536" 
  [193] "17MAG200051536"  "17MAG200051536"  "17MAG200051537"  "17MAG200051537" 
  [197] "17MAG200051537"  "17MAG200051538"  "17MAG200051538"  "17MAG200051538" 
  [2

In [134]:
# Count number of progenies per sire
w=aggregate(y_RR ~ sub, data = df3, FUN=function(x) length(x))[,2]
w # number of messurement par animal par week            
summary(w)

[1]  3  3  3  3  3  3  3  3  3  3  2  2  2  2  1  1  2  2  2  2  1  1  1  1
  [25]  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  [49]  2  2  3  3  3  3  3  3  3  2  3  3  2  2  2  2  2  2  2  2  2  2  2  2
  [73]  3  3  3  3  3  3  3  2  3  2  3  3  3  3  3  3  3  3  1  2  1  2  2  2
  [97]  2  2  2  2  1  1  1  1  1  1  1  1  1  1  2  1  1  1  2  2  2  2  2  1
 [121]  1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  1  1  1  1
 [145]  1  1  1  1  1  1  3  2  3  3  3  3  3  3  3  3  2  2  2  2  2  2  2  2
 [169]  2  2  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  2  2
 [193]  1  1  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
 [217]  2  2  2  1  3  3  3  3  3  3  2  3  1  3  1  1  1  1  1  1  2  2  2  2
 [241]  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
 [265]  1  2  2  1  2  2  2  2  2  2  1  2  3  3  3  3  3  3  3  3  3  3  3  3
 [289]  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  2  3  3  3  3  3  3  3
 [313]  3  3  3  3  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
 [337]  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
 [361]  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
 [385]  3  3  4  4  4  4  4  4  4  4  4  4  1  1  1  1  1  1  1  1  1  1  3  3
 [409]  3  3  3  3  3  3  3  3  2  2  2  2  2  2  2  2  2  2  1  2  1  1  2  2
 [433]  2  2  2  2  3  3  3  3  3  3  3  3  3  3  2  2  2  2  2  2  2  2  2  2
 [457]  1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  2  2  2  2
 [481]  2  2  2  2  2  2  2  1  2  2  1  2  2  2  1  2  3  3  3  3  3  3  3  3
 [505]  3  3  2  2  2  2  2  2  1  1  1  2  1  1  1  1  1  1  1  1  1  1  1  1
 [529]  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2
 [553]  2  2  2  2  2  1  2  2  2  2  2  2  2  2  2  1  2  2  2  2  2  2  2  2
 [577]  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3
 [601]  3  3  3  3  3  3  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
 [625]  2  2  3  3  3  3  3  3  3  3  3  3  1  1  1  1  1  1  1  1  1  1  1  1
 [649]  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  1  1  1  1  1  1
 [673]  1  1  1  1  2  2  2  2  2  2  2  2  2  2  1  1  1  1  1  1  1  1  1  1
 [697]  2  2  2  2  2  2  2  2  2  2  2  1  2  1  1  2  2  1  1  1  2  2  2  2
 [721]  2  1  1  2  2  2  1  1  1  1  1  1  1  1  1  1  2  1  2  2  3  4  3  3
 [745]  2  3  1  1  1  1  1  1  1  1  1  1  3  3  2  2  3  3  3  3  3  3  5  4
 [769]  5  5  5  5  5  5  5  5  2  4  2  3  3  4  4  4  4  4  3  2  3  2  3  3
 [793]  3  3  3  3  2  2  2  2  2  2  2  2  2  2  6  2  6  6  6  6  6  6  6  5
 [817]  3  2  3  3  3  3  3  3  3  3  2  1  2  2  2  2  2  2  2  2  1  1  1  1
 [841]  1  1  1  2  2  2  1  2  2  2  2  2  4  4  4  3  3  3  4  4  4  4  4  4
 [865]  4  4  4  4  4  4  4  4  1  1  1  1  1  1  1  1  1  1  7  5  7  7  7  7
 [889]  6  7  7  7  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
 [913]  2  2  2  2  2  2  2  2  2  2  2  3  2  3  3  3  3  3  3  3  4  3  4  4
 [937]  4  4  4  4  4  4  3  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4
 [961]  4  4  6  3  6  6  7  7  7  7  6  6  1  1  1  1  1  1  1  1  1  1  1  1
 [985]  1  1  1  1  1  1  1  2  1  1  1  2  2  2  3  3  3  3  3  3  3  3  3  3
[1009]  3  3  3  3  3  1  3  3  3  3  4  4  4  4  4  4  4  4  4  4  1  1  1  1
[1033]  1  1  1  1  1  1  2  1  2  2  2  2  2  2  2  2  3  3  3  3  3  3  2  3
[1057]  3  3  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  8  7
[1081]  8  8  8  6  8  7  8  8  7  5  7  7  7  6  6  6  6  7  6  6  6  6  6  6
[1105]  6  6  6  6  3  3  3  3  3  3  3  3  3  3  2  2  2  2  2  2  2  2  2  2
[1129]  5  5  5  5  5  5  4  5  5  5  2  2  2  2  2  2  2  2  2  2  3  3  3  3
[1153]  2  3  3  3  3  3  4  3  4  4  4  4  4  4  4  4  8  8  8  8  8  8  8  8
[1177]  8  8  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  1  1
[1201]  1  1  1  1  1  3  3  3  3  3  3  4  4  4  4  

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   2.000   3.000   4.002   5.000  16.000 

Number of progenies per dam ranges from 1 to 16 (median = 3)

In [135]:
df3$w=rep(w,w)
df4=arrange(df3,mere,week)
df4 <- df4[,-1]
# df4=aggregate(. ~ week, data = df3, FUN=function(x) mean(x))
#df4$animal=df4$mere
names(df4)[names(df4)=="mere"] <- "animal"
df5=arrange(df4,animal,week)

In [136]:
head(df5,50)

,animal,week,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,FI,FCR,ADG,RFI,sub,w
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>
1,17MAG20004838,1,22.9610838,25.18634430,24.6404046,26.0103703,23.32080,24.97700,184.5857,2.720211,67.85714,23.5428077,17MAG200048381,3
2,17MAG20004838,1,31.3151825,33.45946199,33.5006679,34.7206289,31.64300,33.21400,217.9714,2.543000,85.71429,28.7338705,17MAG200048381,3
3,17MAG20004838,1,20.0552671,22.30599003,21.2974330,22.6783984,20.53300,22.20100,166.1000,2.906750,57.14286,22.5641764,17MAG200048381,3
4,17MAG20004838,2,11.0091092,12.25525739,10.9469186,11.3309383,11.25680,12.92100,185.3429,2.531512,73.21429,12.0402075,17MAG200048382,3
5,17MAG20004838,2,9.0405746,10.20157330,9.6849286,9.9359466,9.20550,10.78000,214.6714,2.226222,96.42857,6.2158966,17MAG200048382,3
6,17MAG20004838,2,-1.3242796,-0.06501291,-1.5923593,-1.1913334,-1.01400,0.65300,162.7571,2.398526,67.85714,0.9424486,17MAG200048382,3
7,17MAG20004838,3,12.5031435,12.86136221,11.5560406,10.8630460,12.68341,12.13529,194.6857,2.869053,67.85714,14.0691247,17MAG200048383,3
8,17MAG20004838,3,-4.5226132,-4.24579295,-4.7232886,-5.5402791,-4.44100,-5.07800,212.0143,2.283231,92.85714,-7.0582024,17MAG200048383,3
9,17MAG20004838,3,4.2981793,4.66005064,3.3781458,2.6991529,4.48900,3.94100,181.6000,2.676211,67.85714,6.4110481,17MAG200048383,3


In [137]:
df2_p$animal=as.character(df2_p$animal)
df3_p=select(df2_p,animal,pere,week,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,FI, FCR, ADG, RFI)
df3_p <- arrange(df3_p, pere, week)
head(df3_p,30)
length(unique(df3_p$pere))

,animal,pere,week,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,FI,FCR,ADG,RFI
,<chr>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20012762,17MAG20005073,1,1.0465557,3.1871658,2.8068406,3.8308180,0.8850,2.3790,188.9000,2.115680,89.28571,-4.148953
2,17MAG20012763,17MAG20005073,1,-4.5940807,-2.2642429,-3.3955512,-2.0676741,-4.4110,-2.7154,161.3857,2.259400,71.42857,-7.406603
3,17MAG20012765,17MAG20005073,1,1.9349314,4.0692441,3.2182579,4.2052315,1.8967,3.3888,176.8429,2.357905,75.00000,-2.222528
4,17MAG20013045,17MAG20005073,1,-14.6731971,-11.8399807,-13.5030467,-11.5030157,-14.3427,-12.1234,150.9000,2.223789,67.85714,-17.051004
5,17MAG20013047,17MAG20005073,1,-9.7961806,-6.8520022,-9.0250097,-6.8717359,-9.5263,-7.2318,138.1000,2.148222,64.28571,-9.686216
6,8600120012005,17MAG20005073,1,-12.6696022,-10.2009226,-11.4034532,-9.6584347,-12.2640,-10.3604,172.9143,2.200727,78.57143,-11.059817
7,8600120012010,17MAG20005073,1,23.8725077,26.2008665,25.2416442,26.7626567,24.2940,26.0640,185.9571,2.740421,67.85714,24.021439
8,8600120012141,17MAG20005073,1,-23.2448726,-21.2326159,-20.4348102,-19.6168308,-23.1428,-21.6880,179.2143,1.930000,92.85714,-26.534611
9,8600120012157,17MAG20005073,1,-17.2796207,-15.3259006,-14.4255812,-13.6845540,-17.1030,-15.6900,164.7000,2.005043,82.14286,-20.076007


[1] 80

In [138]:
# Count number of progenies per sire

df3_p$sub <- paste0(df3_p$pere,df3_p$week)
df3_p <- arrange(df3_p,sub)
df3_p <- arrange(df3_p,pere,week)
N=aggregate(y_RR ~ sub, data = df3_p, FUN=function(x) length(x))[,2]
df3_p$w=rep(N,N)

head(df3_p)
            
summary(N)
            
# Number of progenies per sire ranges from 9 to 51 (median = 26)

,animal,pere,week,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,FI,FCR,ADG,RFI,sub,w
,<chr>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>
1,17MAG20012762,17MAG20005073,1,1.046556,3.187166,2.806841,3.830818,0.8850,2.3790,188.9000,2.115680,89.28571,-4.148953,17MAG200050731,11
2,17MAG20012763,17MAG20005073,1,-4.594081,-2.264243,-3.395551,-2.067674,-4.4110,-2.7154,161.3857,2.259400,71.42857,-7.406603,17MAG200050731,11
3,17MAG20012765,17MAG20005073,1,1.934931,4.069244,3.218258,4.205232,1.8967,3.3888,176.8429,2.357905,75.00000,-2.222528,17MAG200050731,11
4,17MAG20013045,17MAG20005073,1,-14.673197,-11.839981,-13.503047,-11.503016,-14.3427,-12.1234,150.9000,2.223789,67.85714,-17.051004,17MAG200050731,11
5,17MAG20013047,17MAG20005073,1,-9.796181,-6.852002,-9.025010,-6.871736,-9.5263,-7.2318,138.1000,2.148222,64.28571,-9.686216,17MAG200050731,11
6,8600120012005,17MAG20005073,1,-12.669602,-10.200923,-11.403453,-9.658435,-12.2640,-10.3604,172.9143,2.200727,78.57143,-11.059817,17MAG200050731,11


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   9.00   19.00   26.00   26.51   31.00   51.00 

In [139]:
df4_p <- df3_p[,-1]
names(df4_p)[names(df4_p)=="pere"] <- "animal"
#rename(df4_p, c("pere"="animal")) # rename pere to animal
df4_p=arrange(df4_p,animal,week)

In [140]:
head(df4_p,60)

,animal,week,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,FI,FCR,ADG,RFI,sub,w
,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>
1,17MAG20005073,1,1.0465557,3.1871658,2.80684060,3.8308180,0.88500,2.37900,188.9000,2.115680,89.28571,-4.1489531,17MAG200050731,11
2,17MAG20005073,1,-4.5940807,-2.2642429,-3.39555120,-2.0676741,-4.41100,-2.71540,161.3857,2.259400,71.42857,-7.4066035,17MAG200050731,11
3,17MAG20005073,1,1.9349314,4.0692441,3.21825790,4.2052315,1.89670,3.38880,176.8429,2.357905,75.00000,-2.2225282,17MAG200050731,11
4,17MAG20005073,1,-14.6731971,-11.8399807,-13.50304666,-11.5030157,-14.34270,-12.12340,150.9000,2.223789,67.85714,-17.0510036,17MAG200050731,11
5,17MAG20005073,1,-9.7961806,-6.8520022,-9.02500969,-6.8717359,-9.52630,-7.23180,138.1000,2.148222,64.28571,-9.6862161,17MAG200050731,11
6,17MAG20005073,1,-12.6696022,-10.2009226,-11.40345320,-9.6584347,-12.26400,-10.36040,172.9143,2.200727,78.57143,-11.0598166,17MAG200050731,11
7,17MAG20005073,1,23.8725077,26.2008665,25.24164420,26.7626567,24.29400,26.06400,185.9571,2.740421,67.85714,24.0214385,17MAG200050731,11
8,17MAG20005073,1,-23.2448726,-21.2326159,-20.43481020,-19.6168308,-23.14280,-21.68800,179.2143,1.930000,92.85714,-26.5346107,17MAG200050731,11
9,17MAG20005073,1,-17.2796207,-15.3259006,-14.42558120,-13.6845540,-17.10300,-15.69000,164.7000,2.005043,82.14286,-20.0760065,17MAG200050731,11


In [141]:
df6_p=select(df4_p,animal,animal,week,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,FI, FCR, ADG, RFI,w)
df6  =select(df5,animal,animal,week,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,FI, FCR, ADG, RFI,w)

df6_p$l=2 # from sire
df6$l=1 # from dam

In [142]:
head(df6)

,animal,week,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,FI,FCR,ADG,RFI,w,l
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,17MAG20004838,1,22.961084,25.18634430,24.640405,26.010370,23.3208,24.977,184.5857,2.720211,67.85714,23.5428077,3,1
2,17MAG20004838,1,31.315182,33.45946199,33.500668,34.720629,31.6430,33.214,217.9714,2.543000,85.71429,28.7338705,3,1
3,17MAG20004838,1,20.055267,22.30599003,21.297433,22.678398,20.5330,22.201,166.1000,2.906750,57.14286,22.5641764,3,1
4,17MAG20004838,2,11.009109,12.25525739,10.946919,11.330938,11.2568,12.921,185.3429,2.531512,73.21429,12.0402075,3,1
5,17MAG20004838,2,9.040575,10.20157330,9.684929,9.935947,9.2055,10.780,214.6714,2.226222,96.42857,6.2158966,3,1
6,17MAG20004838,2,-1.324280,-0.06501291,-1.592359,-1.191333,-1.0140,0.653,162.7571,2.398526,67.85714,0.9424486,3,1


In [143]:
head(df6_p)

,animal,week,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,FI,FCR,ADG,RFI,w,l
,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,17MAG20005073,1,1.046556,3.187166,2.806841,3.830818,0.8850,2.3790,188.9000,2.115680,89.28571,-4.148953,11,2
2,17MAG20005073,1,-4.594081,-2.264243,-3.395551,-2.067674,-4.4110,-2.7154,161.3857,2.259400,71.42857,-7.406603,11,2
3,17MAG20005073,1,1.934931,4.069244,3.218258,4.205232,1.8967,3.3888,176.8429,2.357905,75.00000,-2.222528,11,2
4,17MAG20005073,1,-14.673197,-11.839981,-13.503047,-11.503016,-14.3427,-12.1234,150.9000,2.223789,67.85714,-17.051004,11,2
5,17MAG20005073,1,-9.796181,-6.852002,-9.025010,-6.871736,-9.5263,-7.2318,138.1000,2.148222,64.28571,-9.686216,11,2
6,17MAG20005073,1,-12.669602,-10.200923,-11.403453,-9.658435,-12.2640,-10.3604,172.9143,2.200727,78.57143,-11.059817,11,2


In [144]:
# combine df6 with df_p
df7=rbind(df6,df6_p)
df8=arrange(df7,animal,week)
head(df8)
length(unique(df8$animal))

,animal,week,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,FI,FCR,ADG,RFI,w,l
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,17MAG20004838,1,22.961084,25.18634430,24.640405,26.010370,23.3208,24.977,184.5857,2.720211,67.85714,23.5428077,3,1
2,17MAG20004838,1,31.315182,33.45946199,33.500668,34.720629,31.6430,33.214,217.9714,2.543000,85.71429,28.7338705,3,1
3,17MAG20004838,1,20.055267,22.30599003,21.297433,22.678398,20.5330,22.201,166.1000,2.906750,57.14286,22.5641764,3,1
4,17MAG20004838,2,11.009109,12.25525739,10.946919,11.330938,11.2568,12.921,185.3429,2.531512,73.21429,12.0402075,3,1
5,17MAG20004838,2,9.040575,10.20157330,9.684929,9.935947,9.2055,10.780,214.6714,2.226222,96.42857,6.2158966,3,1
6,17MAG20004838,2,-1.324280,-0.06501291,-1.592359,-1.191333,-1.0140,0.653,162.7571,2.398526,67.85714,0.9424486,3,1


[1] 624

In [145]:
head(DF_s4)

,animal,pe2,week,identif,bande,sex,lignee,gener,loge,eleve,...,y_RR,y_RRH,SEgA,y_SAD,SEgH,y_SADH,SEg_MTA,y_MT,SEg_MTH,y_MTH
,<fct>,<int>,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20005070,1,2,17MAG20005070,200011,2,n,G0,71,17MAG,...,94.97610,95.87767,8.846,92.79172,8.403,92.78072,7.612,95.4500,6.915,96.640
2,17MAG20005070,2,4,17MAG20005070,200011,2,n,G0,71,17MAG,...,79.52834,79.07920,7.179,78.04357,7.064,76.62459,5.936,80.2270,5.822,79.649
3,17MAG20005070,2,5,17MAG20005070,200011,2,n,G0,71,17MAG,...,-11.37910,-12.41159,7.112,-11.63063,7.190,-13.23762,5.936,-10.8730,5.822,-11.471
4,17MAG20005070,3,6,17MAG20005070,200011,2,n,G0,71,17MAG,...,-28.62189,-30.16971,7.418,-29.23046,7.678,-31.02945,6.505,-28.2649,6.555,-30.280
5,17MAG20005070,3,7,17MAG20005070,200011,2,n,G0,71,17MAG,...,-13.24151,-15.24268,8.000,-14.00499,8.401,-16.34498,6.505,-13.1429,6.555,-15.175
6,17MAG20005070,3,8,17MAG20005070,200011,2,n,G0,71,17MAG,...,-24.36453,-26.74102,8.805,-25.16910,9.251,-27.83010,6.505,-24.3549,6.555,-26.390


In [146]:
cory1=select(DF_s4,animal,week,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,FI, FCR, ADG, RFI)
cory1=arrange(cory1,animal,week)
cory1$l=3 #mark data obtained from corrected phenotypes
cory1$w=1
df9=rbind(df8,cory1)

In [147]:
head(df9)

,animal,week,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,FI,FCR,ADG,RFI,w,l
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20004838,1,22.961084,25.18634430,24.640405,26.010370,23.3208,24.977,184.5857,2.720211,67.85714,23.5428077,3,1
2,17MAG20004838,1,31.315182,33.45946199,33.500668,34.720629,31.6430,33.214,217.9714,2.543000,85.71429,28.7338705,3,1
3,17MAG20004838,1,20.055267,22.30599003,21.297433,22.678398,20.5330,22.201,166.1000,2.906750,57.14286,22.5641764,3,1
4,17MAG20004838,2,11.009109,12.25525739,10.946919,11.330938,11.2568,12.921,185.3429,2.531512,73.21429,12.0402075,3,1
5,17MAG20004838,2,9.040575,10.20157330,9.684929,9.935947,9.2055,10.780,214.6714,2.226222,96.42857,6.2158966,3,1
6,17MAG20004838,2,-1.324280,-0.06501291,-1.592359,-1.191333,-1.0140,0.653,162.7571,2.398526,67.85714,0.9424486,3,1


In [231]:
fix(cory1)

In [148]:
dim(df9)
length(unique(df9$animal))

[1] 65772    14

[1] 2951

In [149]:
#ped_all <- read.csv("pedigreeRFIall", header=F, sep="")
head(ped_all,20)

,V1,V2,V3,V4,V5,V6
,<int>,<fct>,<fct>,<fct>,<fct>,<fct>
1,127,n,G0,17MAG20004838,53DH319994424,17M3019991718
2,128,n,G0,17MAG20004841,53DH319994424,17M3019991718
3,129,n,G0,17MAG20004842,53DH319994424,17M3019991718
4,130,n,G0,17MAG20004965,79QEK19986393,17M3019984533
5,131,n,G0,17MAG20005004,50CPQ19989292,17M3019973048
6,132,n,G0,17MAG20005005,50CPQ19989292,17M3019973048
7,133,n,G0,17MAG20005070,79MFY19982545,17M3019997935
8,134,n,G0,17MAG20005071,79MFY19982545,17M3019997935
9,135,n,G0,17MAG20005072,79MFY19982545,17M3019997935


In [232]:

#df9=merge(cory1,df8,c("animal","week","y_RR","y_RRH","y_SAD","y_SADH","y_MT","y_MTH","FI", "FCR", "ADG", "RFI","w","l"),all=T)
df10=arrange(df9,animal,week)

#df10 = df8 #here we ignoring the owning performances
ped_all <- read.csv("pedigreeRFIall", header=F, sep="")
names(ped_all)<-c("N","lignee","gener", "animal","p","m")
df_all=merge(ped_all,df10,c("animal"))
df_all1=arrange(df_all,animal,week)

In [236]:
tail(df_all1)
fix(df_all1)

,animal,N,lignee,gener,p,m,week,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,FI,FCR,ADG,RFI,w,l
,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
65767,8600120117378,6413,+,G7,17MAG20081422,8600120097047,4,16.37112,16.63697,16.23699,15.85298,15.860,16.301,213.0143,2.385760,89.28571,16.52536,1,3
65768,8600120117378,6413,+,G7,17MAG20081422,8600120097047,5,23.92424,23.37190,23.68804,22.78104,22.734,23.170,234.7429,2.528000,92.85714,23.48711,1,3
65769,8600120117378,6413,+,G7,17MAG20081422,8600120097047,6,39.14358,37.80628,38.92044,37.53043,39.672,37.277,255.6714,2.753385,92.85714,37.73355,1,3
65770,8600120117378,6413,+,G7,17MAG20081422,8600120097047,7,30.97324,28.90813,31.35018,29.23018,31.056,28.660,267.8429,2.586069,103.57143,28.21755,1,3
65771,8600120117378,6413,+,G7,17MAG20081422,8600120097047,8,23.84710,21.12140,24.32604,21.71005,23.647,21.260,272.5286,2.543600,107.14286,20.69702,1,3
65772,8600120117378,6413,+,G7,17MAG20081422,8600120097047,9,32.53427,29.19715,31.82023,28.38024,31.318,27.610,294.5714,2.660645,110.71429,28.88808,1,3


In [237]:
df_all2=select(df_all1,animal,week,gener,lignee,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,FI, FCR, ADG, RFI,w,l)
df_all2=arrange(df_all2,animal,week,w)

In [245]:
fix(df_all2)

In [260]:
df_all2$animal=as.character(df_all2$animal)
df_all2$week=as.numeric(df_all2$week)
df_all2$lignee=as.character(df_all2$lignee)
df_all2$gener=as.character(df_all2$gener)
df_all3=aggregate(y_RR ~animal+week+w+gener+lignee+l, data = df_all2, mean)
df_all3$y_RRH=aggregate(y_RRH ~animal+week+w+gener+lignee+l, data = df_all2, mean)[,7]
df_all3$y_SAD=aggregate(y_SAD ~animal+week+w+gener+lignee+l, data = df_all2, mean)[,7]
df_all3$y_SADH=aggregate(y_SADH ~animal+week+w+gener+lignee+l, data = df_all2, mean)[,7]
df_all3$y_MT=aggregate(y_MT ~animal+week+w+gener+lignee+l, data = df_all2, mean)[,7]
df_all3$y_MTH=aggregate(y_MTH ~animal+week+w+gener+lignee+l, data = df_all2, mean)[,7]
df_all3$y_FI=aggregate(FI ~ animal+week+w+gener+lignee+l, data = df_all2, mean)[,7]
df_all3$y_FCR=aggregate(FCR ~animal+week+w+gener+lignee+l, data = df_all2, mean)[,7]
df_all3$y_RFI=aggregate(RFI ~animal+week+w+gener+lignee+l, data = df_all2, mean)[,7]
df_all3$y_ADG=aggregate(ADG ~animal+week+w+gener+lignee+l, data = df_all2, mean)[,7]


df_all4=arrange(df_all3,animal,week)


# remove replicated rows
library(dplyr)
df_all5 <- df_all4 %>%
  group_by(animal, week) %>% 
  filter(row_number() == 1)                     
                     


write.table(df_all5,"DATA_corrected.txt")


In [261]:
head(df_all5,100)


animal,week,w,gener,lignee,l,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,y_FI,y_FCR,y_RFI,y_ADG
<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
17MAG20004838,1,3,G0,n,1,24.7771778,26.9839321,26.479502,27.80313253,25.165600,26.7973333,189.5524,2.723320,24.946952,70.23810
17MAG20004838,2,3,G0,n,1,6.2418014,7.4639393,6.346496,6.69185051,6.482767,8.1180000,187.5905,2.385420,6.399518,79.16667
17MAG20004838,3,3,G0,n,1,4.0929032,4.4252066,3.403633,2.67397327,4.243805,3.6660967,196.1000,2.609498,4.473990,76.19048
17MAG20004838,4,3,G0,n,1,1.4517633,0.9697848,1.348915,-0.16375756,1.409471,0.8314300,210.1905,2.411680,1.079648,88.09524
17MAG20004838,5,3,G0,n,1,6.7104368,5.4808968,7.297009,5.47743377,6.518805,5.9377633,230.8619,2.317436,6.037846,98.80952
17MAG20004838,6,3,G0,n,1,18.8087020,16.9323528,18.576286,16.45908494,18.907367,16.3870000,247.4667,2.659138,18.623236,94.64286
17MAG20004838,7,3,G0,n,1,6.1774453,3.7406692,6.071266,3.29802397,6.176933,3.6625333,243.2952,2.571481,5.884617,94.04762
17MAG20004838,8,3,G0,n,1,-7.8175510,-10.7259968,-8.723363,-11.85230973,-7.817733,-10.3204667,220.2952,2.650300,-6.348409,83.33333
17MAG20004838,9,3,G0,n,1,-20.0090692,-23.3214693,-20.037852,-23.29338320,-19.598333,-22.9998000,226.1143,2.452647,-19.841687,91.66667


In [262]:
tttt = df_all5[duplicated(df_all5[,c("animal","week")]),]
head(tttt)

animal,week,w,gener,lignee,l,y_RR,y_RRH,y_SAD,y_SADH,y_MT,y_MTH,y_FI,y_FCR,y_RFI,y_ADG
<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


In [263]:
length(unique(df_all5[df_all5$gener =='G7',]$animal))

[1] 528

In [264]:
length(unique(df_all5[df_all5$gener =='G6',]$animal))

[1] 456

In [259]:
length(unique(df_all5[df_all5$gener =='G5',]$animal))

[1] 417

# Computing the Accuracies
## Random regression models

pheno <- read.table("DATA_corrected.txt", header=TRUE, quote="\"")
pheno=arrange(pheno,gener,animal)
pheno$cd=as.numeric(pheno$gener)
pheno=subset(pheno,pheno$cd>=6)


#l==1 , corrected missing phenotype,l==2 corrected phenotypes
#Here ,if RFI+ put "+" else "-"
pheno=subset(pheno,lignee=="+")
sln <- read.table("sln_RRAp",quote="\"") 


names(sln)[1]<-"kk"  
names(sln)[2]<-"a"  
names(sln)[3]<-"para"  
names(sln)[4]<-"sdpara"  



a = sln$a
elems <- unlist( strsplit(as.character(a)  , "\\." ) )
m <- as.data.frame(matrix( elems , ncol = 2 , byrow = TRUE ))
sln=cbind(sln,m)
names(sln)<-c("kk","a","para","SE","gr","animal")




sln$para1=ifelse(sln$gr==1,sln$para,0)  
sln$para2=ifelse(sln$gr==2,sln$para,0) 
sln$para3=ifelse(sln$gr==3,sln$para,0)  
sln=arrange(sln,animal,gr)  


# taking all animals 2993


# sln1=merge(sln,my2,by="animal")
sln1=sln

sln1=arrange(sln1,animal,gr)


tatemp <- read.table("OP.txt", quote="\"", comment.char="") 
names(tatemp)[1]<-"week"  
names(tatemp)[2]<-"f1"  
names(tatemp)[3]<-"f2"  
names(tatemp)[4]<-"f3"  

T=t(as.matrix(tatemp[2:4]))

SLN=sln1[,c("animal","gr","para")]
SLN1=arrange(SLN,animal,gr)

SLN2=reshape(SLN1,varying = NULL, timevar = "gr",idvar = "animal",direction="wide",sep="")
SLN3=arrange(SLN2,animal)

SLN4=SLN3[,2:4]

SLN5=as.matrix(SLN4)

SLN5=SLN5




SLN_CD=sln1[,c("animal","gr","SE")]
SLN1_CD=arrange(SLN_CD,animal,gr)
SLN2_CD=reshape(SLN1_CD,varying = NULL, timevar = "gr",idvar = "animal",direction="wide",sep="")
SLN3_CD=arrange(SLN2_CD,animal)
SLN4_CD=SLN3_CD[,2:4]
SLN5_CD=as.matrix(SLN4_CD)


SE_CD <- matrix(0,dim(SLN5_CD)[1],10)
r01 <- 0.3619 
r02 <- 0.7141E-01
r12 <- 0.1586 








sigma_A <- diag(as.matrix(read.table("C:/Users/HP/OneDrive/RFI_all/RFI_all/G_OPA", quote="\"", comment.char="")))

for (i in (1:dim(SLN5_CD)[1])){
  # a <- SLN4_CD[i,1]*SLN4_CD[i,1]
  # b <- SLN4_CD[i,2]*SLN4_CD[i,2]
  # c <- SLN4_CD[i,3]*SLN4_CD[i,3]
  
  a <- SLN4_CD[i,1]*SLN4_CD[i,1]
  b <- SLN4_CD[i,2]*SLN4_CD[i,2]
  c <- SLN4_CD[i,3]*SLN4_CD[i,3]
  
  
  for (j in (1:10))
  {
    w0 <- T[1,j]
    w1 <- T[2,j] 
    w2 <- T[3,j] 
    SE_CD[i,j] <- a*(w0*w0) + b*(w1*w1) +  c*(w2*w2) +
      2*(w0*w1*r01*sqrt(a*b) + w0*w2*r02*sqrt(a*c) + w1*w2*r12*sqrt(b*c))
    
  }
  
}





#SECD_all=as.data.frame((SLN5_CD)%*%(T))

for (i in (1:3986)){
  SE_CD[i,] <- SE_CD[i,]/sigma_A
  
}

colMeans(SE_CD)

sqrt(colMeans(SE_CD))

#colvar(SE_CD)
 
for (j in (1 : dim(SE_CD)[2])){
   print(var(SE_CD[,j]))
}
          






EBV_all=as.data.frame((SLN5)%*%(T))

colnames(EBV_all) <- c("V1","V2","V3","V4","V5","V6","V7","V8","V9","V10")

EBV_all$animal=SLN2$animal

EBV_all1=melt(EBV_all, id = "animal",measure.vars = c("V1","V2","V3","V4","V5","V6",
                                                      "V7","V8","V9","V10"))
EBV_all2=arrange(EBV_all1,animal,variable)
EBV_all2$EBV=EBV_all2$value

#EBV_all2$week=rep(c(1:10),81)
#EBV_all2$week=rep(c(1:10),2993)
EBV_all2$week=rep(c(1:10),3986)


mydata_all = EBV_all2
mydata_all$week=rep(c(1:10),length(mydata_all$animal)/10)

#mydata_all= merge(EBV_all2,my2,by=c("animal"))

mydata_all1=arrange(mydata_all,animal,week)

filename<-paste("EBV_all_RRA",".txt", sep="")  
write.table(mydata_all1,filename)  

#mydf=merge(mydata_all1,pheno3,by=c("animal","week","gener","lignee"))

mydf=merge(mydata_all1,pheno,by=c("animal","week"))
mydf1=arrange(mydf,animal,week)











for (i in (1:10)){
  ll =subset(mydf1, week == i )
  # out = wtd.cor(ll$EBV,ll$y_RR,ll$w)/sqrt(h2_A[i])
  out = wtd.cor(ll$EBV,ll$y_RR)
  print(var(ll$EBV))
  #print(out)
  # print(out/sqrt(h2_A[i]))
}




list_g <- read.table("list", quote="\"")
names(list_g)= c("N","animal") 
# 
 mydf2=merge(mydf1,list_g,c("animal"))
mydf3H=arrange(mydf2,animal,week)

#Here, if calculate for dam, put the below line, else then hide it

#mydf3H=subset(mydf3H,l==1) # if dam

mydf3H1=mydf3H


for (i in (1:10)){
  
  ll =subset(mydf3H, week == i)
  #out = with(ll,wtd.cor(EBV,y_RRH,w))/sqrt(h2_RRH[i])
  out = with(ll,wtd.cor(EBV,y_RR,w))
  cat("----week----",i,"------", '\n')
  #print(var(ll$EBV))
  #print(out/sqrt(h2_RRH[i]) )
  
}


mydf3H1=aggregate(mydf3H1,list(mydf3H1$animal,mydf3H1$week),tail,1)
 mydf3H1$y_RR=aggregate(mydf3H$y_RR,list(mydf3H$animal,mydf3H$week),mean)[,3]
 mydf3H1$y_RRH=aggregate(mydf3H$y_RRH,list(mydf3H$animal,mydf3H$week),mean)[,3]

mydf3H=mydf3H1






# with(mydf3H,lm(y_RRH~EBV))


with(mydf3H,lm(y_RR~EBV,weight=w))
summary(with(mydf3H,lm(y_RR~EBV,weight=w)))


h2_RRA = c(0.19831238, 0.12573177, 0.06854276, 0.06487875, 0.07596792, 0.08953578,
           0.09016391, 0.14022587, 0.17514397, 0.23679283
           
)






for (i in (1:10)){
  
  ll =subset(mydf3H, week == i)
  out = with(ll,wtd.cor(EBV,y_RR,w))
   out1 = with(ll,wtd.cor(EBV,y_RR,w))/sqrt(round(h2_RRA[i],2))
  
 out2 = with(ll,wtd.cor(EBV,y_RR,w))/sqrt(colMeans(SE_CD))[i]
  
  #out = with(ll,wtd.cor(EBV,y_RRH,w))
  cat("----week----",i,"------", '\n')
 print(out)
  #print(out1)
# print(out2) 
  
}




for (i in (1:10)){
  
  mydf4H=subset(mydf3H,week==i)
  out = summary(with(mydf4H,lm(y_RR~EBV,weight=w)))$coefficients 
  
  
  
  cat("----week----",i,"------", '\n')
  print (out[2,1]-0.5)
  print(out[2,])
  
}









################################################################################
















G_RR <- as.matrix(read.table("C:/Users/HP/OneDrive/RFI_all/RFI_all/G_OPA", quote="\"", comment.char=""))




DD <- arrange(mydf3H, animal, week)





week <- c(1:10)
vec2 <- eigen(G_RR)$vectors[,2]
vec1 <- eigen(G_RR)$vectors[,1]

eigen_dat <- cbind.data.frame(week,vec1,vec2)
DD1 <- merge(DD, eigen_dat, by = "week")

DD2 <- arrange(DD1, animal, week)


DD2$y_RR1 <- DD2$vec1 * DD2$y_RR
DD2$y_RR2 <- DD2$vec2 * DD2$y_RR

DD2$EBV_RR1 <- DD2$vec1 * DD2$EBV
DD2$EBV_RR2 <- DD2$vec2 * DD2$EBV








DD2 <- arrange(DD2,animal,week)



library(dplyr)
DD3 <- DD2 %>%
  group_by(animal) %>%
  mutate(y_RR1_SBV = sum(y_RR1, na.rm = TRUE), 
         y_RR2_SBV = sum(y_RR2, na.rm = TRUE),
         SBV1 = sum(EBV_RR1, na.rm = TRUE),
         SBV2 = sum(EBV_RR2, na.rm = TRUE)
         
  )



DD4<- aggregate(DD3,list(DD3$animal,DD3$gener,DD3$lignee),mean,na.rm=TRUE)




DD4$animal <-DD4$Group.1
DD4$gener <- DD4$Group.2
DD4$lignee <- DD4$Group.3

DD4 <- DD4[,c("animal","lignee","gener","w",
              "y_RR"   ,     "y_RRH"    ,   "y_SAD"   ,    "y_SADH"  ,    "y_MT"  ,     
              "y_MTH"     ,  "y_FI"     ,   "y_FCR"   ,    "y_RFI" ,      "y_ADG"   ,    
              "y_RR1_SBV"  ,   "y_RR2_SBV"   ,  "SBV1" ,"SBV2")]
 
with(DD4, wtd.cor(y_RR1_SBV, SBV1,w))/sqrt(0.6524008)

with(DD4, wtd.cor(y_RR2_SBV, SBV2,w))/sqrt(0.5925981)



with(DD4, lm(y_RR1_SBV~SBV1,weight = w))$coefficients 



with(DD4, lm(y_RR2_SBV~SBV2,weight = w))$coefficients 



plot(DD4$y_RR1_SBV,DD4$SBV1,  col = DD4$lignee , xlab = "Sy*1", ylab = "SBV1" )


plot(DD4$y_RR2_SBV,DD4$SBV2,  col = DD4$lignee , xlab = "Sy*2", ylab = "SBV2" )








DD5 <- DD4 %>% select( "y_RR1_SBV", "y_RR2_SBV", "SBV1"    , 
                       "SBV2"     )



#DD5_save <- DD5







DD5_save$SBV1p <- DD5$SBV1


DD5_save$SBV2p <- DD5$SBV2



DD5_save1 <- DD5_save[,c(1,2,3,5,4,6)]


wtd.cor(DD5_save1,weight = DD4$w)




save(DD5_save1, file = "DD5_RRm.Rdata")

